In [1]:
from __future__ import print_function
import pyrealsense2.pyrealsense2 as rs
import numpy as np
import cv2
import time
import os
import datetime
import skimage
import tensorflow as tf
import pandas as pd
# import mercury
# import RPi.GPIO as GPIO
# import multiprocessing as mp
# GPIO.setwarnings(False)
# GPIO.setmode(GPIO.BCM)
# GPIO.setup(13, GPIO.IN)
# from myFunctions import *

In [2]:
def read_tag(power):
    reader = mercury.Reader("tmr:///dev/ttyUSB0", baudrate=115200)
    reader.set_region("NA2")
    reader.set_read_plan([1], "GEN2", bank=["user"], read_power=power)
    tag = reader.read(timeout=500)
    if len(tag)!=0:
        tag = str(bytes.fromhex(reader.read()[0].epc.decode('utf-8')))
        tag = tag[2:-1]
    if len(tag) == 0:
        tag = 'No_RFID'

    return tag

def image_preprocessing(depth_img, background):
    depth_img = background - depth_img

    otsu_thresh = skimage.filters.threshold_otsu(depth_img)
    img_post = np.where(depth_img<otsu_thresh, depth_img, 0)

    img_post = np.where(img_post>600, depth_img, 0)

    img_mask = np.where(img_post != 0, 1, 0)
    img_seg = skimage.measure.label(img_mask, background = 0, connectivity=2)
    assert( img_seg.max() != 0 )
    mask = img_seg == np.argmax(np.bincount(img_seg.flat)[1:])+1

    depth_img = np.where(mask != 0, depth_img, 0)

    return depth_img, mask


In [3]:
fps = 30
width = 848
length = 480
total_pixels = width*length


# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file('March_Data\\Cow_15_68.bag')
#config.enable_stream(rs.stream.depth, width, length, rs.format.z16, fps)
#config.enable_stream(rs.stream.color, width, length, rs.format.bgr8, fps)

pipe_profile = pipeline.start(config)

#decimation = rs.decimation_filter()
#decimation.set_option(rs.option.filter_magnitude, 2)

spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 2)

thresh = rs.threshold_filter(min_dist = 0.1, max_dist = 3.0)

# depth_sensor = pipe_profile.get_device().first_depth_sensor()

# depth_sensor.set_option(rs.option.enable_auto_exposure, True)

#getting the background when the camera is first turned on
frames = pipeline.wait_for_frames()
depth_frame = frames.get_depth_frame()
color_frame = frames.get_color_frame()
thresh_frame = thresh.process(depth_frame)
filtered_depth = spatial.process(thresh_frame)
background_image = np.asanyarray(filtered_depth.get_data())
print(np.shape(background_image))




while True:

    #getting frames from the camera
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    thresh_frame = thresh.process(depth_frame)
    filtered_depth = spatial.process(thresh_frame)
    depth_image = np.asanyarray(filtered_depth.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    #import imageio
    #background_image = imageio.imread('masks\\feb_background.tif')
    cropped_depth, mask = image_preprocessing(depth_image, background_image)
    upper_pixels = np.sum(np.where(cropped_depth[:100, :] != 0, 1, 0))
    lower_pixels = np.sum(np.where(cropped_depth[length-20:length, :] != 0, 1, 0))
    total_nonzero_pixels = np.sum(np.where(cropped_depth != 0, 1, 0))
    cv2.line(cropped_depth,(0,100),(848,100),(255,0,0),1)
    cv2.line(cropped_depth,(0, length-20),(848, length-20),(255,0,0),1)
    
    cv2.putText(cropped_depth, 'Lower Pixel values: '+str(lower_pixels),(0,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255),2,cv2.LINE_AA)
    cv2.putText(cropped_depth, 'Upper Pixel values: '+str(upper_pixels),(0,150), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255),2,cv2.LINE_AA)

    cv2.imshow('cropped depth', cropped_depth)
    cv2.waitKey(1)
    #print(lower_pixels, upper_pixels, total_nonzero_pixels)

    if (lower_pixels < 400) and (upper_pixels > 15000): # and (total_nonzero_pixels > 10000):

        #cropped_depth = (cropped_depth/np.max(cropped_depth)*255).astype('uint8')

        cropped_color = color_image
        cropped_color[:, :, 0] = np.where(mask==True, color_image[:, :, 0], 0)
        cropped_color[:, :, 1] = np.where(mask==True, color_image[:, :, 1], 0)
        cropped_color[:, :, 2] = np.where(mask==True, color_image[:, :, 2], 0)

        # them to be suitable for the ViT
        # cropped_color = skimage.transform.resize(cropped_color, (224, 224, 3))
        # cropped_depth = skimage.transform.resize(cropped_depth, (224, 224))

        #Now convert depth/color images to DGE
        img_DGE = np.zeros(np.shape(color_image))
        img_DGE[:, :, 0] = cropped_depth
        img_DGE[:, :, 1] = skimage.color.rgb2gray(cropped_color)
        mask_dilation = skimage.morphology.binary_dilation(mask, skimage.morphology.ball(4, dtype=bool)[::, ::, 0], out=None)
        img_DGE[:, :, 2] = skimage.feature.canny(cropped_depth, sigma=1.0).astype('uint8')*255*mask_dilation

        cv2.imshow('cropped color', img_DGE)
        cv2.waitKey(1)
    #cv2.destroyAllWindows()

RuntimeError: Failed to resolve request. Request to enable_device_from_file("March_Data\Cow_15_68.bag") was invalid, Reason: Failed to create ros reader: Error opening file: March_Data\Cow_15_68.bag